In [1]:
from snlg_analyses import SnLg_Analyses

# Define the directory paths
BASE_DIRECTORY = './Inputs/'
RAW_DIRECTORY = BASE_DIRECTORY + 'raw/'
CORRECTED_DIRECTORY = BASE_DIRECTORY + 'corrected/'
SELECTED_DIRECTORY = BASE_DIRECTORY + 'selected/'

# Initialize class configuration
CONFIG = {
    "model": 'iasp91',   # Make sure the file exists when reading model from user-defined file. Default: 'iasp91'
    "base_directory": BASE_DIRECTORY,
    "raw_directory": RAW_DIRECTORY,
    "corrected_directory": CORRECTED_DIRECTORY,
    "selected_directory": SELECTED_DIRECTORY,
    "enable_write_sac": True, 
    "enable_archive_waveform": True,
    "enable_remove_resp": True,
    "enable_sanity_check": True,
    "enable_trim_edge": True,
    "edge_trim_length": 10,
    "prefilt_window": (0.05,0.1,10,20),
    "sn_filt":(0.25, 1.0), # originally 1-4
    "lg_filt":(0.25, 1.0), # originally 0.5-4
    "enable_zero_phase_filter": True,
    "min_epi_distance": 350,
    "max_epi_distance": 2000,
    "seconds_before_P": 60,
    "seconds_after_P": 400,
    "noise_windowlen": 20,
    "noise_offset": 5,
    "vsm": 4.7,
    "vsc": 3.7,
    "moho_snlg": 65,
    "snlg_SNR_threshold": 3,
    "enable_overwrite_log": True,
    "enable_new_log"  : False,
    "log_fname": BASE_DIRECTORY + "log_iris",
    "enable_console_output": False,
    "snlglist_filename": "snlg_list_iris",
    "enable_chunk_list": True,
    "max_records_before_dump": 3000
}

SnLg = SnLg_Analyses(**CONFIG)

# Prepare directories
SnLg.prepare_directory(SnLg.raw_directory)
SnLg.prepare_directory(SnLg.corrected_directory)
SnLg.prepare_directory(SnLg.selected_directory)

In [2]:
import pandas as pd
import numpy as np
from obspy.geodetics import kilometers2degrees
# A useful function to reduce the number of stations
# This function is not perfect, please modify it according to your purpose
def select_stations_by_azimuth_distance(
    inventory, evla, evlo, max_station=20, n_azimuth_bins=8, n_distance_bins=8
):
    """
    Select representative stations using 2D binning by azimuth and distance.
    """
    from obspy.geodetics.base import gps2dist_azimuth
    from obspy.core.inventory import Inventory, Network

    station_info = []
    for network in inventory:
        for station in network:
            stla = station.latitude
            stlo = station.longitude
            dist_m, az, _ = gps2dist_azimuth(evla, evlo, stla, stlo)
            dist_km = dist_m / 1000
            dist_deg = kilometers2degrees(dist_km)
            station_info.append({
                'network': network.code,
                'station': station.code,
                'latitude': stla,
                'longitude': stlo,
                'distance_deg': dist_deg,
                'distance_km': dist_km,
                'azimuth': az,
                'station_obj': station
            })

    df = pd.DataFrame(station_info)

    # Bin azimuth [0, 360)
    df['az_bin'] = pd.cut(df['azimuth'], bins=np.linspace(0, 360, n_azimuth_bins + 1), labels=False, include_lowest=True)

    # Bin distance (km)
    dist_edges = np.linspace(df['distance_km'].min(), df['distance_km'].max(), n_distance_bins + 1)
    df['dist_bin'] = pd.cut(df['distance_km'], bins=dist_edges, labels=False, include_lowest=True)

    # Select top 1 station per (az_bin, dist_bin), or more depending on max_station
    selected_stations = []
    for _, group in df.groupby(['az_bin', 'dist_bin']):
        group_sorted = group.sort_values(by='distance_km')
        selected_stations.extend(group_sorted.head(1).to_dict('records')) # head(N) -> the first N stations

    # Limit to max_station
    selected_stations = selected_stations[:max_station]

    # Reconstruct a trimmed inventory
    new_networks = {}
    for s in selected_stations:
        net_code = s['network']
        if net_code not in new_networks:
            new_networks[net_code] = Network(code=net_code, stations=[])
        new_networks[net_code].stations.append(s['station_obj'])

    return Inventory(networks=list(new_networks.values()), source=inventory.source)


In [3]:
from obspy import UTCDateTime
from get_seismic_waveforms import get_waveform_from_client
from obspy.clients.fdsn import Client
import pandas as pd
import time

MAXIMUM_STATION = 40
# client = Client("IRIS")
client = Client("IRIS", timeout=60)
s_time = time.perf_counter()

# Read Inventory and Catalog
cata_df = pd.read_csv('./Inputs/Tibet.csv',index_col=False)
# print(type(cata_df))

min_dis_deg = kilometers2degrees(CONFIG["min_epi_distance"])
max_dis_deg = kilometers2degrees(CONFIG["max_epi_distance"])

print(min_dis_deg,max_dis_deg)

for idx, row in cata_df.iterrows():
    # 1) Make a single-event DataFrame
    #    The double brackets [[idx]] ensure it's still a DataFrame, not a Series
    event_df = cata_df.loc[[idx]]
    # 2) Build the per-event inventory.
    #    e.g., selecting stations within some radius of this event
    # print(row)
    evla = row.event_lat
    evlo = row.event_lon
    starttime = UTCDateTime(row.event_time)
    endtime   = starttime + 3600
    # print(type(starttime))
    # TEST CODE: use BH* stations for simplicity ...
    for attempt in range(2):
        try:
            custom_inventory = client.get_stations(
            level='channel',    # level='channel' is nessary for rotating BH[12] -> BH[NE]
            starttime=starttime, endtime=endtime,
            channel='BH*', network="*", location="*", station="*",
            latitude=evla, longitude=evlo,
            minradius=min_dis_deg, maxradius=max_dis_deg
        )
            break
        except Exception as e:
            logger0.info(f"[Worker {worker_id}] Attempt {attempt+1} failed: {e}")
            time.sleep(1)
    else:
        continue  # skip this event
        
    if sum(len(net.stations) for net in custom_inventory) > MAXIMUM_STATION:
        custom_inventory = select_stations_by_azimuth_distance(
            inventory=custom_inventory,
            evla=evla,
            evlo=evlo,
            max_station=MAXIMUM_STATION,
            n_azimuth_bins=8,
            n_distance_bins=8
        )

    # print(custom_inventory)
    SnLg.process_snlg_bulk(catalog_dataframe=event_df, inventory=custom_inventory, 
                  get_seismic_trace_func=get_waveform_from_client,
                  client=client,channel_requested='BH*',location_requested="*")
    
SnLg.output_final_snlg_list()

e_time = time.perf_counter()
elapsed_time = (e_time - s_time) / 60
print(f"Elapsed time: {elapsed_time:.4f} minutes")


3.147625620715557 17.986432118374612


 WARNING (norm_resp): computed and reported sensitivities differ by more than 5 percent. 
	 Execution continuing.
 WARNING (norm_resp): computed and reported sensitivities differ by more than 5 percent. 
	 Execution continuing.
 WARNING (norm_resp): computed and reported sensitivities differ by more than 5 percent. 
	 Execution continuing.
 WARNING (norm_resp): computed and reported sensitivities differ by more than 5 percent. 
	 Execution continuing.
 WARNING (norm_resp): computed and reported sensitivities differ by more than 5 percent. 
	 Execution continuing.
 WARNING (norm_resp): computed and reported sensitivities differ by more than 5 percent. 
	 Execution continuing.
 WARNING (norm_resp): computed and reported sensitivities differ by more than 5 percent. 
	 Execution continuing.
 WARNING (norm_resp): computed and reported sensitivities differ by more than 5 percent. 
	 Execution continuing.
 WARNING (norm_resp): computed and reported sensitivities differ by more than 5 percent.

Elapsed time: 4.1525 minutes
